In [ ]:
import torch
from torch.utils.data import DataLoader
import lava.lib.dl.slayer as slayer
from main import NMNIST, SNN, augment

In [ ]:
dir = '.'
nmnist = NMNIST(dir, 1, 300, download=True, transform=augment)
nmnist_test = NMNIST(dir, 1, 300, train=False)

In [ ]:
device = torch.device('cpu')

model = SNN()
#state_dict = torch.load('network.pt')
#model.load_state_dict(state_dict)

net = model.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

nmnist_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

nmnist_test_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

error = slayer.loss.SpikeRate(
    true_rate=0.2, false_rate=0.03, reduction='sum'
).to(device)

stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(
    net, error, optimizer, stats,
    classifier=slayer.classifier.Rate.predict
)

epochs = 2

for epoch in range(epochs):
    for i, (input, label) in enumerate(nmnist_loader):
        output = assistant.train(input, label)
        header = [
            'TRAIN\n' +
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    for i, (input, label) in enumerate(nmnist_test_loader):
        output = assistant.test(input, label)
        header = [
            'TEST\n' +
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    torch.save(net.state_dict(), dir + '/network.pt')

    stats.update()

In [ ]:
model = SNN()
state_dict = torch.load('network.pt')
model.load_state_dict(state_dict)

In [ ]:
nmnist_test.example_of_each_class(model, 4)